ASSUMPTIONS FOR DATASET:
3 DataFrames:
- NUmber of Periods Alloted Per Week for Each Class
- For Each Subject and Class Allotted Teachers and HOME ROOM Teachers
- HOME ROOM Teachers Allotted for particular classes (eligible to teach all subjects of that class without loss of generality)

The Data Tables must be well organised

In [162]:
!pip install -q -U weasyprint gradio_client

In [172]:
from pandas import read_csv, concat, to_numeric
from weasyprint import HTML, CSS

## Class for Oragnising Data recieved from Files

In [164]:
class Standards:

    def __init__(self, periodAllotment: DataFrame, teacherAllotment: DataFrame, hRTeacherAllotment: DataFrame):
        self.__standards = dict()
        standards = list(std.replace(" ", "").lower() for std in teacherAllotment.columns.tolist()[1:])
        hRTeacherAllotment['standard'] = hRTeacherAllotment['standard'].apply(lambda x: x.lower().replace(" ", ''))
        hRTeacherAllotment.set_index("standard", inplace = True)
        for standard in standards:
            self.__standards[standard] = self.__initialisingStandard(standard, periodAllotment, teacherAllotment, hRTeacherAllotment)
        print(self.__standards.keys())

    def __initialisingStandard(self, name: str, periodAllotment: DataFrame, teacherAllotment: DataFrame, hRTeacherAllotment: DataFrame) -> str:
        for column in periodAllotment.columns[1:]:
            if column.replace(" ", "").lower() in name:
                periodAllotment = periodAllotment[['Subject', column]]
        periodAllotment = periodAllotment[periodAllotment.apply(lambda x: to_numeric(x, errors='coerce').notna().any(), axis=1)]
        periodAllotment['Subject'] = periodAllotment['Subject'].str.lower().str.replace(" ", '')
        periodAllotment.set_index("Subject", inplace = True)
        periodAllotment.columns = ['Number of Periods per week']
        std = list(s for s in teacherAllotment.columns.tolist()[1:] if s.replace(" ", '').lower() == name)[0]
        teacherAllotment = teacherAllotment[['Subject', std]]
        teacherAllotment['Subject'] = teacherAllotment['Subject'].str.lower().str.replace(" ", '')
        teacherAllotment.dropna(inplace = True)
        teacherAllotment.set_index("Subject", inplace = True)
        teacherAllotment.loc[teacherAllotment[std] == "HR", std] = hRTeacherAllotment.loc[name, "Home room Teachers "]
        return concat([periodAllotment, teacherAllotment], join = "outer", axis = 1).to_markdown(index = True)

    def stringify(self):
        for std, data in self.__standards.items():
            yield std, data

## Laoding Files and Preprocessing

In [165]:
periodAllotment = read_csv("/content/Period Allotment.csv")
teacherAllotment = read_csv("/content/Teacher-Subject Mapping.csv")
hRTeacherAllotment = read_csv("/content/Teacher Subject Mapping 2.csv")

In [166]:
periodAllotment

,Subject,Jr. KG,Sr. KG,I
0,Eng,5.0,5.0,8.0
1,Hindi,4.0,4.0,6.0
2,Mathematics,4.0,4.5,7.0
3,Marathi,NaN,NaN,2.0
4,EVS,2.0,3.0,5.0
5,Computer Studies,0.5,0.5,2.0
6,Circle Time,0.5,0.5,1.0
7,Life Skills,0.5,0.5,1.0
8,Physical Education,2.0,2.0,2.0
9,Yoga,0.5,0.5,1.0


In [167]:
teacherAllotment

,Subject,Jr.KG A,Jr.KG B,Jr.KG C,Jr.KG D,Sr.KG A,Sr.KG B,Sr.KG C,Sr.KG D,I A,I B,I C,I D
0,Eng,"Nupur FT , Pooja R","Maria , Banu","Shireen , Riya J",Sheetal Anjum,Ruksaar Darshana,Bhakti Laxmipriya,"Shubha FT , Priyanka",Ankita\n Jyoti V,"Remya,Saraswati -FT","Kanak,Matilda","Rishita,Shashwati,","Charu ,Maariyah,"
1,Hindi,HR,HR,HR,HR,HR,HR,HR,HR,Reshma,Reshma,Santosh K,Santosh K
2,Marathi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dakshata,Dakshata,Pratibha,Pratibha
3,Mathematics,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR
4,EVS,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR
5,Environmental Education,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR
6,Computer Studies,Sayantani,Sayantani,Sayantani,Sayantani,Sayantani,Sayantani,Sayantani,Sayantani,Trupti,Sayantani,Sayantani,Sayantani
7,Physical EDUCATION,Naval,Naval,Beena,Beena,"Priyanka.A, Beena","Priyanka.A, Beena",Beena,Naval,ARJUN,ARJUN,Priyanka.A,Naval
8,Circle Time,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR,HR
9,Life skills,Vanita J,Vanita J,Vanita J,Vanita J,Vanita J,Vanita J,Vanita J,Vanita J,Zamora,Zamora,Zamora,Zamora


In [168]:
hRTeacherAllotment

,standard,Home room Teachers,Subjects
0,Jr.KG A,"Kim,Rukhsaar,",All
1,Jr.KG B,"Maria, Shital",All
2,Jr.KG C,"Shireen , Uphaar",All
3,Jr.KG D,"Priyanka ,Darshana",All
4,Sr.KG A,"Pooja,Nupur,",All
5,Sr.KG B,Anjum Lakshmipriya,All
6,Sr.KG C,"Shubha , Jyoti",All
7,Sr.KG D,"Ankita,Bhakti,",All
8,I A,"Remya,Saraswati -FT",All
9,I B,"Kanak,Matilda",All


## Creating Object

In [147]:
standards_data = Standards(periodAllotment, teacherAllotment, hRTeacherAllotment)

dict_keys(['jr.kga', 'jr.kgb', 'jr.kgc', 'jr.kgd', 'sr.kga', 'sr.kgb', 'sr.kgc', 'sr.kgd', 'ia', 'ib', 'ic', 'id'])


## Prompt Generation

In [169]:
def prompt(std, start_time, end_time, period_duration, lunch_breaks, assembly, data):
    return f"""Generate a school timetable for **{std}** using the following constraints:
Each subject's name is written in Pascal Case with proper spaces (e.g., 'Life Skills', 'Mathematics', 'Physical Education').
**Constraints**:
- Start Time: {start_time}
- End Time: {end_time}
- Period Duration: {period_duration}
- Lunch Breaks:
  - {lunch_breaks}
- Assembly: {assembly}
- School Days: Monday to Friday (5 days a week)
- Subjects and Weekly Periods:
{data}
Write subject and teacher name in each cell. If a teacher is not available for a subject, write only subject name. All the constraint should match with the final Timetable about number ofperiods alotted to each subject and there teachers. If float number is given in NUmber of Periods, that means the Periods should be interpreted as integer for respective number of weeks. For example 0.5 periods per week means 1 period per 2 week. So if 2 periods have such number can alot 2 periods in the same cell.
**Output Format**:
Provide only the timetable as a table in html format, with Heading of the class. Exclude any introductory or explanatory text. Format the output with rows for days (Monday to Friday) and columns for periods, including subject name, teacher name, and timing in each cell."""

In [170]:
# @title Input School Details
start_time = "Jr.Kg, Sr.Kg: 10:15 AM 1st: 8:15 AM" # @param {'type': 'string'}
end_time = "Jr.Kg, Sr.Kg: 2:15 PM 1st: 2:15 PM" # @param {'type': 'string'}
period_duration = "30 mins" # @param {'type': 'string'}
lunch_breaks = "Jr.Kg, Sr.Kg: 12:45 - 1:15 PM  1st: 9:25 - 9:45 AM, 12:45 - 1:15 PM" # @param {'type': 'string'}
assembly = "Jr.Kg, Sr.Kg: Tuesday, 10:45AM 1st: Tuesday, 10:45AM" # @param {'type': 'string'}

## Creating Time Tables

In [173]:
%%capture
css = CSS(string="""
        @page {
            size: A4 landscape;
            margin: 1cm;
        }
        body {
            font-family: Arial, sans-serif;
            font-size: 12px;
            line-height: 1.5;
        }
        table {
            width: 100%;
            border-collapse: collapse;
        }
        th, td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: center;
        }
        th {
            background-color: #f2f2f2;
        }
    """)
for std, data in standards_data.stringify():
    HTML(string = client.predict(query = prompt(std, start_time, end_time, period_duration, lunch_breaks, assembly, data), history = [], system = "", api_name="/model_chat_1")[1][0][-1]).write_pdf(f"{std}.pdf", stylesheets = [css])

DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.006s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.014s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f